Kendall Brown 
8564403
Pstat 134 HW3
Spring 2018

In [ ]:
import pandas as pd
import numpy as np
import pickle
import sklearn.decomposition as skld
import matplotlib.pyplot as plt
import helper_basketball as h
import imp
from ipywidgets import interact, Dropdown, interact_manual
imp.reload(h);

In [ ]:
X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))
X.shape

Q1. Visualiziing the Bases

In [ ]:
def non_negative_marix_decomp(n_components,train_data):
    model = skld.NMF(n_components=n_components, init='nndsvda', max_iter=500, random_state=0)
    W = model.fit_transform(train_data)
    H = model.components_
    nmf = (W,H)
    return(nmf)
xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

In [ ]:
def plot_vectors(nc):
    model = skld.NMF(n_components=nc, init='nndsvda', max_iter=500, random_state=0)
    W = model.fit_transform(X)
    H = model.components_
    fig, ax = plt.subplots(int(nc/2), 2, figsize=(20,40))
    for i, axi in enumerate(ax.flatten()):
        h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
        axi.set_title('NMF component ' + str(i))

In [ ]:
plot_vectors(10)

In [ ]:
plot_vectors(16)

In [ ]:
plot_vectors(20)

In [ ]:
plot_vectors(26)

In [ ]:
plot_vectors(30)

From the above plots we observe a number of different shooting patterns. Even up to 30 bases we observe unique shooting patterns. However, the frequency of non-unique shooting patterns increases quite substantionally. Thus, I take 20-30 bases to be  adequate for reconstructing shooting patterns. 

In [ ]:
W_3,H_3 = non_negative_marix_decomp(n_components = 3,train_data = X)
W_20,H_20 = non_negative_marix_decomp(n_components = 20,train_data = X)
X3_hat = np.matmul(W_3,H_3)
X20_hat = np.matmul(W_20,H_20)

It can be discovered through the all_smooth dictionary in Lab05 that the player numbers 109 and 14 represnt Stephan Curry and Lebron James. We will now reconstruct the shooting patterns of these two players. (They make a lot of layups)

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(20,60))
h.plot_shotchart(X[:,109], xedges, yedges, ax=ax[0,0]) 
h.plot_shotchart(X3_hat[:,109], xedges, yedges, ax=ax[0,1])
h.plot_shotchart(X20_hat[:,109], xedges, yedges, ax=ax[0,2])
ax[0,0].set_title('Original Shooting Pattern')
ax[0,1].set_title('Estimated Shooting Pattern (r=3)')
ax[0,2].set_title('Estimated Shooting Pattern (r=20)')
h.plot_shotchart(X[:,14], xedges, yedges, ax=ax[1,0]) 
h.plot_shotchart(X3_hat[:,14], xedges, yedges, ax=ax[1,1])
h.plot_shotchart(X20_hat[:,14], xedges, yedges, ax=ax[1,2])
ax[1,0].set_title('Original Shooting Pattern')
ax[1,1].set_title('Estimated Shooting Pattern (r=3)')
ax[1,2].set_title('Estimated Shooting Pattern (r=20)')

As expected we clearly see the higher base order leads to a much better approximation of the original shooting pattern. We can calculate the error rate of each base number.

In [ ]:
from numpy import linalg as LA 
print('Error rate of base 3',LA.norm(X-np.matmul(W_3,H_3),'fro'))
print('Error rate of base 20',LA.norm(X-np.matmul(W_20,H_20),'fro'))

We observe an error rate of ~18.7% and ~11.5% for bases 3 and 20 respectively. However, we should be more interested in how many bases we need to consider for minimal error rate.

I am going to comment out the next chunk of code as it takes quite a while to run. Results are given in the following paragraph.

In [ ]:
#r_values = np.arange(30)+1
#error_norm = []
#for r in r_values[20:]:
#    W,H = non_negative_marix_decomp(n_components = r,train_data = X)
#    error_norm.append(LA.norm(X-np.matmul(W,H),'fro'))

In [ ]:
#error_norm

In [ ]:
#plt.plot(r_values[20:],error_norm)

Above we see the error rate as a function of the number of bases. We notice that an increase to 30 bases results yields a 1% improvement of the error rate. Furthermore we can expect to see significant diminishing returns as we increase the number of bases. Considering the time to calculate the error rate associated with 30 bases, and how little of an improvement was achieved, I conclude that, with the resources at my disposal, it is simply not worth the effort to go beyond a ~90% accuracy level    

Q2. Calculating Correlation Matrix, number of dimensions should equal number of players.

In [ ]:
X_corr=X.copy()
R = np.corrcoef(X_corr,rowvar=0)

In [ ]:
print('Dimensions of R',R.shape)
x=(R[R<=1])
x.sort()
x_min=x[[1,3]]
x_max=x[[362**2-363,362**2-365]]
max_ind=np.argwhere((R==x_max[0]) | (R==x_max[1]))
min_ind=np.argwhere((R==x_min[0]) | (R==x_min[1]))
print('Max correlation of',x_max,'between observations',max_ind[0:2],
      '\nMin correlation of',x_min,'between observations',min_ind[0:2])

Heatmap of matrix correlation matrix R. As expected we see perfect correlation and symetry about the diagonal. Pockets of high and low correlation lay outside the diaonal allong well defined lines distingushing the most and least correlated players.

In [ ]:
import seaborn as sns; sns.set()
ax = sns.heatmap(R)

Shooting patterns of the most correlated players.

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(20,60))
h.plot_shotchart(X[:,101], xedges, yedges, ax=ax[0,0]) 
h.plot_shotchart(X[:,243], xedges, yedges, ax=ax[0,1]) 
h.plot_shotchart(X[:,235], xedges, yedges, ax=ax[1,0]) 
h.plot_shotchart(X[:,283], xedges, yedges, ax=ax[1,1]) 
ax[0,0].set_title('Shooting Pattern of Player 101')
ax[0,1].set_title('Shooting Pattern of Player 243')
ax[1,0].set_title('Shooting Pattern of Player 235')
ax[1,1].set_title('Shooting Pattern of Player 283')

Shooting pattern of least correlated players.

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(20,60))
h.plot_shotchart(X[:,21], xedges, yedges, ax=ax[0,0]) 
h.plot_shotchart(X[:,28], xedges, yedges, ax=ax[0,1]) 
h.plot_shotchart(X[:,23], xedges, yedges, ax=ax[1,0]) 
h.plot_shotchart(X[:,251], xedges, yedges, ax=ax[1,1]) 
ax[0,0].set_title('Shooting Pattern of Player 21')
ax[0,1].set_title('Shooting Pattern of Player 28')
ax[1,0].set_title('Shooting Pattern of Player 23')
ax[1,1].set_title('Shooting Pattern of Player 251')

Heirachical Clustering according to shooting patterns. Because we are clustering a correlation matrix, we take the euclidian distance metric calculated with clusters decided by the average distance between observations. 

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

In [ ]:
Z = linkage(R,method='average',metric='euclidean')
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90,
    leaf_font_size=2.,
)
plt.show()

As observed from the matrix heatmap and the above dendrogram. A rather large portion of the players lay within a single cluster. There are only a few distinct groups with only a few elements being clustered outside of the main cluster. Implying that, in general, there are only a few distinct shooting patterns with a single pattern being much more common than the others. Based on the vector plots above, I'm going to assume that NBA players strongly prefer layups and shots close to the hoop.

Q3. When factoring the matrix X, we created two matrices W and H. Here the matrix W represents each unique bin a shot may be taken from. Since all players can theoretically shoot from each bin, we can express their shooting style as a linear 
combination of all unique shooting styles. Naturally, we assume that each player has their own preferred shooting pattern. Thus, per player, we assign coefficients to each unique shooting bin. These coefficients are the elements of matrix H and represent a player's unique predisposition towards shooting from a particular bin. In short, a players shooting pattern can be expressed as $\sum\limits_{i=0}^n {H_i*W_i}$, and the larger the value of $H_i$ the more likely a player is to take a shot from bin $W_i$. A player can have a shooting pattern which can be described as a subset composed of $H_i$ values. Where the values of the subset's elements must be above a certain significance value. In short if the set $H_i$-$H_j$ represents the likelihood of shooting from the three-point line, a player can be classifined as a three-point shooter if their unique $H_i$-$H_j$ have elements above a certain threshold. 

In the case of problem 2 we, we see players being clustered into groups according to their correlated shooting patterns. We can infer that the clustering groups were decided based on similarities in the linear combination representing their shooting patterns. Since $W_i$ is a variable representing the shooting bin, we must assume that if any similarities arise between players it must be the result of having similar $H_i$ values. In other words, if a group of players prefer to take a specific type of layup, their respective H matricies would have a relatively large $H_i$ value associated with that layup bin $W_i$.

This coincides with the article linked in the assignment notebook file, where players were clustered according to a classification label assigned to them by their observed shooting pattern. Here players are clustered according to their correlation with one another. Since players that are highly correlated with eachother have similar shooting patterns. We must assume that players within the same cluster share similar shooting patterns, and thus similar H values. 

Q4. Philosophically speaking, as a team manager, the shooting pattern of a player is critical information in the basketball meta game. In a hypothetical situation, where I am responsible for creating a lineup that best counters the opposing team's lineup, I would like to have the analysis of the opposing players' shooting and blocking patterns and then design a lineup such that the preffered shooting and blocking patterns of the offensive lineup is in opposition towards the opposing team's play pattern. With this lineup optimally chosen such that the chosen players are direct counter-picks to the opposing lineup, the desired outcome of victory should be quite easily achieved.   

Obviously, there are more in-depth correlations I would need to address. For instance, intuitively we can expect some sort of interaction between players (the play pattern of player A may be directly corelated to player B's prescence on the court). Is this between player interaction significant and does it change the shotting pattern or success rate of each player? What happens when a player starts adopting a different playstyle and transitions from one type of player to another over the course of the game. These are important questions which bear significant consequences in deciding the outcome of a game.       

This takes the game to a different level of strategy where lineup composition could very well be the deciding factor in determining game outcomes. From my personal expirience in playing strategy games where counter-picks and team synergy are a major contributors to victory or defeat. I would say to ignore the potential of a properly picked lineup is to admit defeat before the game even begins.  